# Steering Angle Prediction using Keras
This project is to build a deep neural network to predict steering angle.<br>
Data set: use the <a href="https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip">sample data for track 1</a> provided by Udacity.

## Download data set

In [ ]:
from urllib.request import urlretrieve
from os.path import isfile

if not isfile('data.zip'):
    urlretrieve('https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip','data.zip')

print('data downloaded.')

# unzip the data
import zipfile
if not isfile('data'):
    with zipfile.ZipFile("data.zip","r") as zip_ref:
        zip_ref.extractall()
        
print("file unzipped")



## Parse the log file

In [ ]:
# parse the csv file and pack into pickle for future use
import csv
from os.path import isfile
import pickle

if not isfile("log.p"):
    log = []
    with open('data/driving_log.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        # skip the first row
        reader.__next__()
        # loop through all entry
        for line in reader:
            record = []
            record.append("data/"+line[0].strip())
            record.append("data/"+line[1].strip())
            record.append("data/"+line[2].strip())
            record.append(float(line[3]))
            log.append(record)
    with open('log.p', 'wb') as file:
        pickle.dump(log, file)
        print("log saved")
else:
    with open('log.p', 'rb') as file:
        log = pickle.load(file)
        print("log loaded")
    


## Data augmentation and Data generator
1.random shift<br>
2.random brightness<br>
3.random shadow<br>
4.random flip<br>

In [ ]:
# data augmentation helper functinos
# functions borrow from Vivek Yadav's post
# https://chatbotslife.com/using-augmentation-to-mimic-human-driving-496b569760a9#.ub3zjjxme
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

# random shift
def random_shift(image,angle,trans_range):
    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = 40*np.random.uniform()-40/2
    angle = angle + tr_x/trans_range*2*0.2
    rows, cols, chan = image.shape
    M = np.float32([[1,0,tr_x],[0,1,tr_y]])
    image = cv2.warpAffine(image, M, (cols,rows))
    return image, angle

# random brightness
def random_brightness(image):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = 0.2+np.random.uniform() # range from 0.2 to 1.2 of original image brightness
    image[:,:,2] = image[:,:,2]*random_bright
    image = cv2.cvtColor(image,cv2.COLOR_HSV2RGB)
    return image

# random shadow
def random_shadow(image):
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]
    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    #random_bright = .25+.7*np.random.uniform()
    if np.random.randint(2)==1:
        random_bright = 0.5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)
    return image

# random flip
def random_flip(image, angle):
    flip = np.random.randint(2)
    if flip:
        image = cv2.flip(image,1)
        angle = -angle
    return image, angle

# img_reader
def img_reader(record_id):
    cam_pos = np.random.randint(3)
    image_path = log[record_id][cam_pos]
    # left camera shift angle 0.27, right camera shift angle -0.27
    if (cam_pos == 0):# center cam
        shift_ang = 0
    if (cam_pos == 1): # left cam
        shift_ang = 0.27
    if (cam_pos == 2): # right cam
        shift_ang = -0.27
    angle = file_path = log[record_id][3] + shift_ang
    image = mpimg.imread(image_path)
    
    # augmentation pipeline
    image = random_shadow(image)
    image, angle = random_shift(image, angle, 100)
    image = random_brightness(image)
    image, angle = random_flip(image, angle)
    return image, angle


# test case
record_id = np.random.randint(len(log))
path = log[record_id][0]
print(log[record_id][3])
image = mpimg.imread(path)
plt.figure(0)
plt.imshow(image)
image, angle = img_reader(record_id)
image = image[50:-25,:,:]
print(image.shape)
plt.figure(1)
plt.imshow(image)
print(angle)
plt.figure(2)
plt.imshow(image[:,:,0], cmap="gray")
plt.figure(3)
plt.imshow(image[:,:,1], cmap="gray")
plt.figure(4)
plt.imshow(image[:,:,2], cmap="gray")

In [ ]:
# data generator
def data_generator(log , batch_size = 250):
    batch_images = np.zeros((batch_size, 160, 320, 3))
    batch_angle = np.zeros(batch_size)
    while 1:
        pr_threshold = 0
        while(pr_threshold < 0.4):
            pr_threshold = np.random.uniform()
        for i in range(batch_size):
            record_id = np.random.randint(len(log))      
            keep_pr = 0
            while keep_pr == 0:
                image, angle = img_reader(record_id)
                if abs(angle)<0.1:
                    pr_val = np.random.uniform()
                    if pr_val > pr_threshold:
                        keep_pr = 1
                else:
                    keep_pr = 1
            batch_images[i] = image
            batch_angle[i] = angle
        yield batch_images, batch_angle

def val_generator(log , batch_size = 200):
    batch_images = np.zeros((batch_size, 160, 320, 3))
    batch_angle = np.zeros(batch_size)
    while 1:
        for i in range(batch_size):
            angle = 0
            while abs(angle)<0.1:
                record_id = np.random.randint(len(log)) 
                cam_pos = 0
                image_path = log[record_id][cam_pos]
                angle = file_path = log[record_id][3]
                image = mpimg.imread(image_path)
            image, angle = random_flip(image, angle)
            batch_images[i] = image
            batch_angle[i] = angle
        yield batch_images, batch_angle
        
# test case
gen = data_generator(log)
val_gen = val_generator(log, batch_size = 250)
print("test pass")

## Build Model Graph
The model is nearly identical to the Nvidia paper, except in the paper they convert the color space to YUV. Here we only use RGB.<br>
Thanks to the tips on udacity formus.

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.advanced_activations import ELU
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from keras.regularizers import l1, l2, activity_l2
from keras import backend as K
from keras.layers.core import Lambda


def resize(image_data):
    import tensorflow as tf
    return tf.image.resize_images(image_data, (66, 200))

model = Sequential()
model.add(Cropping2D(cropping=((45, 25), (0, 0)), input_shape=(160, 320, 3), name="cropping"))
model.add(Lambda(lambda x: x/127.5 - 1.0, name="normalize"))
model.add(Lambda(resize, name="resize"))
model.add(Convolution2D(3, 1, 1, init='he_normal', border_mode='valid'))
model.add(Convolution2D(24, 5, 5, activation='elu', init='he_normal', border_mode='valid', subsample=(2,2)))
model.add(Convolution2D(36, 5, 5, activation='elu', init='he_normal', border_mode='valid', subsample=(2,2)))
model.add(Convolution2D(48, 5, 5, activation='elu', init='he_normal', border_mode='valid', subsample=(2,2)))
model.add(Convolution2D(64, 3, 3, activation='elu', init='he_normal', border_mode='valid', subsample=(1,1)))
model.add(Convolution2D(64, 3, 3, activation='elu', init='he_normal', border_mode='valid', subsample=(1,1)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(100, activation='elu', init='he_normal'))
model.add(Dense(50, activation='elu', init='he_normal'))
model.add(Dense(10, activation='elu', init='he_normal'))
model.add(Dense(1))

model.summary() 

## Optimization
Train and save the model.(the models are saved in a folder name "model")<br>
use Adam optimizer with 0.0001 learning rate.<br>
use mse as loss function.<br>
training process is conducted on the destop with GeForce GTX 960

In [ ]:
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from shutil import rmtree
from os import makedirs
from os.path import exists

# remove the old saved model and create new model folder
if exists("./model"):
    rmtree("./model")
    print("removed old model folder")

makedirs("./model")
print("created new model folder")

# save the training result
import json
from keras.models import model_from_json
# model.save_weights("model.h5", True)
with open("model/model.json","w") as file:
    json.dump(json.loads(model.to_json()), file)
print("model.json saved")

# open log file
with open('log.p', 'rb') as file:
    log = pickle.load(file)
    print("log loaded")

# create call backs to store weights after each epcoh
filepath = "model/model.{epoch:02d}.h5"
checkpoint = ModelCheckpoint(filepath)
callback_list = [checkpoint]
    
# optimize the model
optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='mse')
gen = data_generator(log, batch_size=400)
val_gen = val_generator(log)
history = model.fit_generator(generator=gen, samples_per_epoch=20000, nb_epoch=100, validation_data=val_gen, nb_val_samples = 3000, verbose=1, callbacks=callback_list)
print("model weight saved")

## Fine tuning
fine tune the model  
1.run this block to load the model for fine tuning (model files must reside in the same directory as this notebook)  
2.run above block to optimize the model (lower the running rate)

In [ ]:
import json
import numpy as np
from keras.models import model_from_json

model_file = "model.json"
weights_file = "model.h5"
with open(model_file, 'r') as jfile:
    model = model_from_json(jfile.read())
    print("model loaded")
model.load_weights(weights_file)
print("weight loaded")

model.summary() 

## Visualize the model
visualize the model and save to image

In [ ]:
# import json
# import numpy as np
# from keras.models import model_from_json
# from keras.utils.visualize_util import plot


# model_file = "model.json"
# weights_file = "model.h5"
# with open(model_file, 'r') as jfile:
#     model = model_from_json(jfile.read())
#     print("model loaded")
# model.load_weights(weights_file)
# print("weight loaded")
# plot(model, to_file='model.png')
# print("model plot")

# from IPython.display import SVG
# from keras.utils.visualize_util import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Analysis
1. create test images
2. take test images to visualize how CNN responses to it.

In [ ]:
## create a test image
## by tuning brightness and adding shadow
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

# read image
path = "images/test_img_original.jpg"
test1 = mpimg.imread(path)
test2 = mpimg.imread(path)
# change brightness
test1 = cv2.cvtColor(test1, cv2.COLOR_RGB2HSV)
test2 = cv2.cvtColor(test2, cv2.COLOR_RGB2HSV)
test1[:,:,2] = test1[:,:,2]*1.3
test2[:,:,2] = test2[:,:,2]*0.05
test1 = cv2.cvtColor(test1,cv2.COLOR_HSV2RGB)
test2 = cv2.cvtColor(test2,cv2.COLOR_HSV2RGB)
# add shadow
test1 = random_shadow(test1)
test2 = random_shadow(test2)
mpimg.imsave("images/test1.jpg", test1)
mpimg.imsave("images/test2.jpg", test2)
print("images saved")

In [ ]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

test1 = mpimg.imread("images/test1.jpg")
test2 = mpimg.imread("images/test2.jpg")
plt.figure(1)
plt.imshow(test1)
plt.figure(2)
plt.imshow(test1[:,:,0], cmap="gray")
plt.figure(3)
plt.imshow(test1[:,:,1], cmap="gray")
plt.figure(4)
plt.imshow(test1[:,:,2], cmap="gray")
plt.figure(5)
plt.imshow(test2)
plt.figure(6)
plt.imshow(test2[:,:,0], cmap="gray")
plt.figure(7)
plt.imshow(test2[:,:,1], cmap="gray")
plt.figure(8)
plt.imshow(test2[:,:,2], cmap="gray")


In [ ]:
from keras import backend as K
import json
from keras.models import model_from_json
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

# load the model
with open("model.json","r") as file:
    model = model_from_json(file.read())
model.load_weights("model.h5")
model.summary()

# output the images
# path = "images/test1.jpg"
# path = "images/test2.jpg"
# path = "images/feature_test.jpg"
path = "images/road.jpg"
test_image = np.array([mpimg.imread(path)])
plt.figure(0)
plt.imshow(test_image[0])
print(test_image.shape)
input_image = test_image
get_layer_output = K.function([model.layers[0].input],[model.layers[5].output])
layer_output = get_layer_output([input_image])[0]
print(layer_output.shape)
fig = plt.figure(figsize=(6,6)) # size must match the layer output size
for i in range(layer_output.shape[-1]):
    ax = fig.add_subplot(12,3,i+1) # layout grid must match the output size
    ax.imshow(layer_output[0,:,:,i], cmap="gray")
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    plt.tight_layout()